In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import csv
from csv import reader

In [2]:
with open('Train_set', 'r') as read_obj:
    csv_reader = reader(read_obj)
    train = list(csv_reader)
train_list = []
for inner_list in train:
    inner_out_list = []
    for string in inner_list:
        inner_out_list.append(int(string))
    train_list.append(inner_out_list)

In [3]:
train_list

[[1, 0, 2, 1, 1, 0, 0, 2, 2, 2, 2, 2, 2, 1, 1],
 [1, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 2, 0],
 [1, 1, 0, 2, 0, 2, 0, 0, 0, 0, 0, 1, 2, 0, 2],
 [2, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0],
 [2, 2, 1, 1, 1, 0, 0, 2, 0, 0, 0, 2, 0, 1, 0],
 [1, 2, 2, 2, 2, 0, 2, 2, 0, 2, 0, 2, 2, 0, 0],
 [0, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 2, 2, 0, 2, 2, 0, 1, 2, 0, 0, 0, 2, 0],
 [1, 0, 2, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0],
 [2, 1, 1, 1, 2, 0, 2, 0, 2, 0, 2, 0, 0, 0, 0],
 [1, 1, 1, 1, 2, 0, 2, 2, 2, 0, 0, 2, 0, 2, 0],
 [1, 1, 0, 2, 2, 2, 0, 0, 2, 2, 0, 0, 0, 0, 1],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 2, 2, 0, 1, 0, 0, 2, 0, 1, 0, 0, 0, 0, 0],
 [1, 1, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 1, 0],
 [2, 1, 1, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2],
 [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2],
 [1, 2, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0],
 [1, 1, 1, 0, 1, 2, 0, 0, 0, 1, 2, 1, 0, 0, 1],
 [2, 1, 1, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0,

# Baum-Welch algorithm for implmentation of HMM

I'm not thoroughly acquaninted with the algorithm and its mathematics. I have used the algorithm from https://github.com/adeveloperdiary/HiddenMarkovModel/tree/master/part3. I've used the updating scheme in "Multiple sequences" section in https://en.wikipedia.org/wiki/Baum%E2%80%93Welch_algorithm to enable it to take multiple such nod sequences and learn better.

In [4]:
def forward(V, a, b, initial_distribution):
    alpha = np.zeros((V.shape[0], a.shape[0]))
    alpha[0, :] = initial_distribution * b[:, V[0]]
    for t in range(1, V.shape[0]):
        for j in range(a.shape[0]):
            alpha[t, j] = alpha[t - 1].dot(a[:, j]) * b[j, V[t]]
    return alpha

def backward(V, a, b):
    beta = np.zeros((V.shape[0], a.shape[0]))
    beta[V.shape[0] - 1] = np.ones((a.shape[0]))
    for t in range(V.shape[0] - 2, -1, -1):
        for j in range(a.shape[0]):
            beta[t, j] = (beta[t + 1] * b[:, V[t + 1]]).dot(a[j, :])
    return beta

# need to make sure that all lists in V are of same length
def baum_welch(V, a, b, initial_distribution, n_iter=100):
    M = a.shape[0]
    T = len(V[0])
    for n in range(n_iter):
        xi = np.zeros((M, M, T - 1))
        for vs in V:
            alpha = forward(vs, a, b, initial_distribution)
            beta = backward(vs, a, b)
            for t in range(T - 1):
                denominator = np.dot(np.dot(alpha[t, :].T, a) * b[:, vs[t + 1]].T, beta[t + 1, :])
                for i in range(M):
                    numerator = alpha[t, i] * a[i, :] * b[:, vs[t + 1]].T * beta[t + 1, :].T
                    xi[i, :, t] += numerator / denominator
        gamma = np.sum(xi, axis=1)
        a = np.sum(xi, 2) / np.sum(gamma, axis=1).reshape((-1, 1))
        gamma = np.hstack((gamma, np.sum(xi[:, :, T - 2], axis=0).reshape((-1, 1))))
        K = b.shape[1]
        denominator = np.sum(gamma, axis=1)
        for l in range(K):
            sumval = 0
            for vs in V:
                sumval += np.sum(gamma[:, vs == l], axis=1)
            b[:,l] = sumval
        b = np.divide(b, denominator.reshape((-1, 1)))
    return {"a":a, "b":b}

0 => Stable action
1 => Up action
2 => Down action

In [5]:
# this is a 3 nod dataset created manually, all of size 8
V = np.array(train_list)

# no of states and actions are chosen as up, down and stable. 
a = np.ones((3, 3))
a = a / np.sum(a, axis=1)

b = np.ones((3,3))
b = b / np.sum(b, axis=1).reshape((-1, 1))

# starts with stable
initial_distribution = np.array((1,0,0))

# print(baum_welch(V, a, b, initial_distribution, n_iter=100))
# abdict contains both the learned parameters -> a and b.
# these will be used for predictions
abdict = baum_welch(V, a, b, initial_distribution, n_iter=100)

In [6]:
def prediction(test_seq, abdict, threshold=0.001):
    initial_distribution = np.array((1,0,0))
    return (forward(test_seq, abdict['a'], abdict['b'], initial_distribution)[-1,0]*pow(10,-16))

Testing on succesful Nod cases 

In [7]:
print(prediction(np.array([1, 2, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]), abdict))
print(prediction(np.array([2, 0, 1, 1, 1, 0, 0, 2, 0, 0, 0, 0, 0, 2, 1]), abdict))
print(prediction(np.array([1, 1, 0, 2, 1, 0, 2, 2, 1, 2, 0, 0, 0, 1, 2]), abdict))
print(prediction(np.array([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 2, 1]), abdict))
print(prediction(np.array([2, 2, 2, 0, 0, 1, 1, 2, 1, 2, 1, 0, 2, 0, 0]), abdict))
print(prediction(np.array([1, 1, 1, 2, 2, 1, 0, 0, 2, 0, 0, 2, 1, 0, 0]), abdict))
print(prediction(np.array([2, 1, 1, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0]), abdict))
print(prediction(np.array([2, 0, 1, 1, 1, 2, 0, 0, 2, 2, 2, 0, 0, 1, 0]), abdict))
print(prediction(np.array([2, 1, 1, 0, 0, 0, 2, 2, 0, 0, 0, 2, 1, 2, 1]), abdict))
print(prediction(np.array([1, 1, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 2]), abdict))
print(prediction(np.array([1, 1, 1, 2, 0, 2, 0, 0, 2, 1, 0, 1, 2, 0, 0]), abdict))
print(prediction(np.array([1, 1, 0, 2, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0]), abdict))
print(prediction(np.array([2, 2, 1, 1, 1, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0]), abdict))

0.9320538249771981
0.6849682133380844
0.7795781929659338
0.7990164223789075
0.43153353964773256
0.9093791019467455
0.4422935224416814
0.5033845049612659
0.503384504961346
0.44229352244236625
0.909379101946732
0.5159360406229139
0.3791624243917697


Testing on Random cases

In [8]:
print(prediction(np.array([2, 1, 2, 1, 0, 2, 0, 2, 2, 1, 2, 0, 2, 1, 2]), abdict))
print(prediction(np.array([1, 2, 2, 1, 2, 1, 1, 0, 2, 2, 0, 1, 2, 1, 2]), abdict))
print(prediction(np.array([0, 2, 2, 1, 2, 2, 1, 0, 0, 2, 1, 2, 0, 0, 2]), abdict))
print(prediction(np.array([1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0]), abdict))
print(prediction(np.array([0, 0, 2, 0, 1, 0, 2, 1, 0, 0, 0, 0, 2, 2, 2]), abdict))
print(prediction(np.array([1, 1, 0, 2, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 1]), abdict))
print(prediction(np.array([2, 2, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 1, 0]), abdict))
print(prediction(np.array([1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0]), abdict))
print(prediction(np.array([1, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), abdict))

0.31713485823119325
0.8872560027078373
0.23887430089907843
0.25093513616016483
0.20988440232873848
0.4422935224421973
0.285595092129033
0.2509351361601763
0.3886165705709504
